# ChangeDotOrgScraper

In [ ]:
# from ChangeDotOrgScraper import scrape_petitions

# # Specify the url of your scrape.org searches
# ## for instance, we shall scrape petitions from the search "supplemental nutrition assistance program"
# url='https://www.change.org/search?q=Supplemental%20Nutrition%20Assistance%20Program&offset=0'

# # scrape the petition data from change.org
# SNAP_petitions_df=scrape_petitions(url)
# print(SNAP_petitions_df)

# HuggingFace models

https://huggingface.co/ariannap22/collectiveaction_roberta_simplified_synthetic_weights

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Set device to CPU or GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_name = "ariannap22/collectiveaction_roberta_simplified_synthetic_weights"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define the text you want to predict
texts = [
    "We need to stand together for our rights!",
    "I volunteer at the local food bank.",
    "This is just a casual discussion.",
    "If he was just like, look, I'll give you a donation, but I don't want to be on stage and shoot because I'm not really an activist. I'm not an activist. Would you have a problem with that?"
]

# Tokenize the input text
inputs = tokenizer(
    texts,
    padding=True,  # Pad to the longest sequence in the batch
    truncation=True,  # Truncate sequences longer than the model's max length
    max_length=512,  # Adjust max length as needed
    return_tensors="pt"  # Return PyTorch tensors
).to(device)

# Perform prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Raw model outputs before softmax

# Convert logits to probabilities (optional)
probs = torch.nn.functional.softmax(logits, dim=-1)

# Get predicted class indices
predicted_class_indices = torch.argmax(probs, dim=-1)

# Print results
for text, idx, prob in zip(texts, predicted_class_indices, probs):
    print(f"Text: {text}")
    print(f"Predicted Class Index: {idx.item()}")
    print(f"Probabilities: {prob.tolist()}")
    print("---")

https://huggingface.co/ariannap22/collectiveaction_roberta_synthetic_weights_layered

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Set device to CPU or GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_name = "ariannap22/collectiveaction_roberta_synthetic_weights_layered"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the text you want to predict
texts = [
    "We need to stand together for our rights!",
    "I volunteer at the local food bank.",
    "If he was just like, look, I'll give you a donation, but I don't want to be on stage and shoot because I'm not really an activist. I'm not an activist. Would you have a problem with that?"
]

# Tokenize the input text
inputs = tokenizer(
    texts,
    padding=True,  # Pad to the longest sequence in the batch
    truncation=True,  # Truncate sequences longer than the model's max length
    max_length=512,  # Adjust max length as needed
    return_tensors="pt"  # Return PyTorch tensors
).to(device)

# Perform prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Raw model outputs before softmax

# Convert logits to probabilities (optional)
probs = torch.nn.functional.softmax(logits, dim=-1)

# Get predicted class indices
predicted_class_indices = torch.argmax(probs, dim=-1)

# Print results
for text, idx, prob in zip(texts, predicted_class_indices, probs):
    print(f"Text: {text}")
    print(f"Predicted Class Index: {idx.item()}")
    print(f"Probabilities: {prob.tolist()}")
    print("---")

https://huggingface.co/ariannap22/collectiveaction_sft_annotated_only_v6_prompt_v6_p100_synthetic_balanced_more_layered

In [ ]:
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          pipeline)

model_dir = "ariannap22/collectiveaction_sft_annotated_only_v6_prompt_v6_p100_synthetic_balanced_more_layered"

# Define the text you want to predict
texts = [
    "We need to stand together for our rights!",
    "I volunteer at the local food bank."
]

# Define levels of participation in collective action¨
dim_def = {'Problem-Solution': "The comment highlights an issue and possibly suggests a way to fix it, often naming those responsible.",
            'Call-to-Action': "The comment asks readers to take part in a specific activity, effort, or movement.",
            'Intention': "The commenter shares their own desire to do something or be involved in solving a particular issue.",
            'Execution': "The commenter is describing their personal experience taking direct actions towards a common goal."}

# Define the prompt
def generate_test_prompt6(data_point):
    return f"""
            You have the following knowledge about levels of participation in collective action that can be expressed in social media comments: {dim_def}. 
            
            ### Definitions and Criteria:
            **Collective Action Problem:** A present issue caused by human actions or decisions that affects a group and can be addressed through individual or collective efforts.

            **Participation in collective action**: A comment must clearly reference a collective action problem, social movement, or activism by meeting at least one of the levels in the list {dim_def.keys()}.

            Classify the following social media comment into one of the levels within the list {list(dim_def.keys())}. 

            ### Example of correct output format:
            text: xyz
            label: None
            
            Return the answer as the corresponding participation in collective action level label.

            text: {data_point}
            label: """.strip()

texts_prompts = [generate_test_prompt6(text) for text in texts]

# Prepare datasets and load model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_dir)

tokenizer.pad_token_id = tokenizer.eos_token_id

# Define prediction 
def predict(texts, model, tokenizer):
    y_pred = []
    answers = []
    categories = list(dim_def.keys())

    for i in range(len(texts)):
        prompt = texts[i]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=20, 
                        temperature=0.1)
        
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        answers.append(answer)
        
        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("error")
    
    return y_pred, answers

y_pred, answer = predict(texts_prompts, model, tokenizer)

# Print results
for text, pred in zip(texts, y_pred):
    print(f"Text: {text}")
    print(f"Predicted Class: {pred}")
    print("---")

# SPoRC dataset
Line Separator is '\n'

In [21]:
import pandas as pd    
episodeLevelData = pd.read_json("../data/episodeLevelDataSample.jsonl", lines=True)
speakerTurnData = pd.read_json("../data/speakerTurnDataSample.jsonl", lines=True)

episodeLevelDataSample

In [ ]:
import sqlite3

# Connect to SQLite3 database (or create it)
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Create a table based on the column names provided
cursor.execute('''
CREATE TABLE IF NOT EXISTS podcast_episodes (
    transcript TEXT,
    rssUrl TEXT,
    epTitle TEXT,
    epDescription TEXT,
    mp3url TEXT,
    podTitle TEXT,
    lastUpdate REAL,
    itunesAuthor TEXT,
    itunesOwnerName TEXT,
    explicit INTEGER,
    imageUrl TEXT,
    language TEXT,
    createdOn REAL,
    host TEXT,
    podDescription TEXT,
    category1 TEXT,
    category2 TEXT,
    category3 TEXT,
    category4 TEXT,
    category5 TEXT,
    category6 TEXT,
    category7 TEXT,
    category8 TEXT,
    category9 TEXT,
    category10 TEXT,
    oldestEpisodeDate TEXT,
    episodeDateLocalized REAL,
    durationSeconds REAL,
    hostPredictedNames TEXT,
    numUniqueHosts REAL,
    guestPredictedNames TEXT,
    numUniqueGuests REAL,
    neitherPredictedNames TEXT,
    numUniqueNeithers REAL,
    mainEpSpeakers TEXT,
    numMainSpeakers REAL,
    hostSpeakerLabels TEXT,
    guestSpeakerLabels TEXT,
    overlapPropTurnCount REAL,
    avgTurnDuration REAL,
    overlapPropDuration REAL,
    totalSpLabels REAL
)
''')

# Commit the changes
conn.commit()

In [ ]:
import json

# Open and read the JSONL file
with open('../data/episodeLevelDataSample.jsonl', 'r') as f:
    first_record = json.loads(f.readline())

# Display the keys of the first record to ensure the fields match
print(first_record.keys())

In [15]:
# Extract relevant fields from the first record
values_to_insert = (
    first_record.get('transcript', None),
    first_record.get('rssUrl', None),
    first_record.get('epTitle', None),
    first_record.get('epDescription', None),
    first_record.get('mp3url', None),
    first_record.get('podTitle', None),
    first_record.get('lastUpdate', None),
    first_record.get('itunesAuthor', None),
    first_record.get('itunesOwnerName', None),
    first_record.get('explicit', None),
    first_record.get('imageUrl', None),
    first_record.get('language', None),
    first_record.get('createdOn', None),
    first_record.get('host', None),
    first_record.get('podDescription', None),
    first_record.get('category1', None),
    first_record.get('category2', None),
    first_record.get('category3', None),
    first_record.get('category4', None),
    first_record.get('category5', None),
    first_record.get('category6', None),
    first_record.get('category7', None),
    first_record.get('category8', None),
    first_record.get('category9', None),
    first_record.get('category10', None),
    first_record.get('oldestEpisodeDate', None),
    first_record.get('episodeDateLocalized', None),
    first_record.get('durationSeconds', None),
    first_record.get('hostPredictedNames', None),
    first_record.get('numUniqueHosts', None),
    first_record.get('guestPredictedNames', None),
    first_record.get('numUniqueGuests', None),
    first_record.get('neitherPredictedNames', None),
    first_record.get('numUniqueNeithers', None),
    first_record.get('mainEpSpeakers', None),
    first_record.get('numMainSpeakers', None),
    first_record.get('hostSpeakerLabels', None),
    first_record.get('guestSpeakerLabels', None),
    first_record.get('overlapPropTurnCount', None),
    first_record.get('avgTurnDuration', None),
    first_record.get('overlapPropDuration', None),
    first_record.get('totalSpLabels', None)
)

# Convert the tuple to a list
values_to_insert = list(values_to_insert)

# Now modify the value at position 28
values_to_insert[28] = ', '.join(values_to_insert[28]) if isinstance(values_to_insert[28], list) else values_to_insert[28]
values_to_insert[34] = ', '.join(values_to_insert[34]) if isinstance(values_to_insert[34], list) else values_to_insert[34]
values_to_insert[36] = json.dumps(values_to_insert[36])  # Convert dictionary to JSON string

# Convert the list back to a tuple after modification
values_to_insert = tuple(values_to_insert)

# Insert the extracted data into the table
cursor.execute('''
    INSERT INTO podcast_episodes (
        transcript, rssUrl, epTitle, epDescription, mp3url, podTitle, lastUpdate, itunesAuthor, 
        itunesOwnerName, explicit, imageUrl, language, createdOn, host, podDescription, 
        category1, category2, category3, category4, category5, category6, category7, 
        category8, category9, category10, oldestEpisodeDate, episodeDateLocalized, 
        durationSeconds, hostPredictedNames, numUniqueHosts, guestPredictedNames, 
        numUniqueGuests, neitherPredictedNames, numUniqueNeithers, mainEpSpeakers, 
        numMainSpeakers, hostSpeakerLabels, guestSpeakerLabels, overlapPropTurnCount, 
        avgTurnDuration, overlapPropDuration, totalSpLabels
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', values_to_insert)

# Commit the changes
conn.commit()

In [ ]:
# for i, val in enumerate(values_to_insert):
#     print(f"Value at position {i}: {val} (type: {type(val)})")

In [ ]:
import sqlite3
from tabulate import tabulate

# Assuming you have an active connection 'conn' to your database
cursor = conn.cursor()

# Execute the SELECT query to fetch all data from the podcast_episodes table
cursor.execute('SELECT * FROM podcast_episodes limit 10')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection when done
cursor.close()
conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the query to count the number of entries
cursor.execute('SELECT COUNT(*) FROM podcast_episodes')
count = cursor.fetchone()[0]

# Print the count
print(f"Number of entries in podcast_episodes: {count}")

# Close the cursor and connection
cursor.close()
conn.close()

In [26]:
# import sqlite3
# import pandas as pd

# # Connect to the database
# conn = sqlite3.connect('data.db')

# # Query the podcast_episodes table
# query = "SELECT * FROM podcast_episodes"
# df = pd.read_sql_query(query, conn)

# # Save the dataframe to a CSV file
# df.to_csv('podcast_episodes.csv', index=False)

# # Close the connection
# conn.close()

In [ ]:
df.head()

In [ ]:
import json
import sqlite3

# Open the JSONL file and connect to the database
with open('../data/episodeLevelDataSample.jsonl', 'r') as f:
    # Open a connection to your database
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()

    # Iterate through each line in the JSONL file
    for line in f:
        record = json.loads(line)

        # Display the keys of the record to ensure the fields match (optional for debugging)
        print(record.keys())  # You can comment this line after confirming fields are correct

        # Extract relevant fields and prepare them for insertion
        values_to_insert = (
            record.get('transcript', None),
            record.get('rssUrl', None),
            record.get('epTitle', None),
            record.get('epDescription', None),
            record.get('mp3url', None),
            record.get('podTitle', None),
            record.get('lastUpdate', None),
            record.get('itunesAuthor', None),
            record.get('itunesOwnerName', None),
            record.get('explicit', None),
            record.get('imageUrl', None),
            record.get('language', None),
            record.get('createdOn', None),
            record.get('host', None),
            record.get('podDescription', None),
            record.get('category1', None),
            record.get('category2', None),
            record.get('category3', None),
            record.get('category4', None),
            record.get('category5', None),
            record.get('category6', None),
            record.get('category7', None),
            record.get('category8', None),
            record.get('category9', None),
            record.get('category10', None),
            record.get('oldestEpisodeDate', None),
            record.get('episodeDateLocalized', None),
            record.get('durationSeconds', None),
            record.get('hostPredictedNames', None),
            record.get('numUniqueHosts', None),
            record.get('guestPredictedNames', None),
            record.get('numUniqueGuests', None),
            record.get('neitherPredictedNames', None),
            record.get('numUniqueNeithers', None),
            record.get('mainEpSpeakers', None),
            record.get('numMainSpeakers', None),
            record.get('hostSpeakerLabels', None),
            record.get('guestSpeakerLabels', None),
            record.get('overlapPropTurnCount', None),
            record.get('avgTurnDuration', None),
            record.get('overlapPropDuration', None),
            record.get('totalSpLabels', None)
        )

        # Convert the tuple to a list for modification
        values_to_insert = list(values_to_insert)

        # Modify the values at positions 28, 34, and 36 as needed
        # Position 28: hostPredictedNames (if it's a list, join it as a string)
        if isinstance(values_to_insert[28], list):
            values_to_insert[28] = ', '.join(values_to_insert[28])
        elif isinstance(values_to_insert[28], dict):
            values_to_insert[28] = json.dumps(values_to_insert[28])  # Convert dict to JSON string

        # Position 34: mainEpSpeakers (if it's a list, join it as a string)
        if isinstance(values_to_insert[34], list):
            values_to_insert[34] = ', '.join(values_to_insert[34])
        elif isinstance(values_to_insert[34], dict):
            values_to_insert[34] = json.dumps(values_to_insert[34])  # Convert dict to JSON string

        # Position 36: hostSpeakerLabels (if it's a dictionary, convert to a JSON string)
        if isinstance(values_to_insert[36], dict):
            values_to_insert[36] = json.dumps(values_to_insert[36])  # Convert dict to JSON string

        # Ensure all values are properly handled as strings or None
        for i in range(len(values_to_insert)):
            if values_to_insert[i] is None:
                values_to_insert[i] = 'NULL'  # SQLite's representation for NULL
            elif isinstance(values_to_insert[i], (list, dict)):
                values_to_insert[i] = str(values_to_insert[i])  # Fallback: convert any lists or other types to string

        # Convert the list back to a tuple after modification
        values_to_insert = tuple(values_to_insert)

        # Insert the record into the database
        cursor.execute('''
            INSERT INTO podcast_episodes (
                transcript, rssUrl, epTitle, epDescription, mp3url, podTitle, lastUpdate, itunesAuthor, 
                itunesOwnerName, explicit, imageUrl, language, createdOn, host, podDescription, 
                category1, category2, category3, category4, category5, category6, category7, 
                category8, category9, category10, oldestEpisodeDate, episodeDateLocalized, 
                durationSeconds, hostPredictedNames, numUniqueHosts, guestPredictedNames, 
                numUniqueGuests, neitherPredictedNames, numUniqueNeithers, mainEpSpeakers, 
                numMainSpeakers, hostSpeakerLabels, guestSpeakerLabels, overlapPropTurnCount, 
                avgTurnDuration, overlapPropDuration, totalSpLabels
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', values_to_insert)

    # Commit the changes to the database
    conn.commit()

    # Close the cursor and connection
    cursor.close()

speakerTurnDataSample

In [ ]:
import json

# Open and read the JSONL file
with open('../data/speakerTurnDataSample.jsonl', 'r') as f:
    first_record = json.loads(f.readline())

# Display the keys of the first record to ensure the fields match
print(first_record.keys())

In [ ]:
print(len(first_record.keys()))

In [ ]:
import sqlite3

# Connect to SQLite3 database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Create the speaker_turns table
try:
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS speaker_turns (
        mfcc1_sma3Mean REAL,
        mfcc2_sma3Mean REAL,
        mfcc3_sma3Mean REAL,
        mfcc4_sma3Mean REAL,
        F0semitoneFrom27_5Hz_sma3nzMean REAL,
        F1frequency_sma3nzMean REAL,
        turnText TEXT,
        speaker TEXT,
        startTime REAL,
        endTime REAL,
        duration REAL,
        mp3url TEXT,
        turnCount INTEGER,
        inferredSpeakerRole TEXT,
        inferredSpeakerName TEXT
    )
    ''')
    
    # Commit changes
    conn.commit()
    print("Table speaker_turns created successfully.")
    
except sqlite3.Error as e:
    print(f"Error creating table: {e}")
    
finally:
    # Close the connection
    conn.close()

In [ ]:
import json
import sqlite3

# Open the JSONL file and connect to the database
with open('../data/speakerTurnDataSample.jsonl', 'r') as f:
    # Open a connection to your database
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()

    # Iterate through each line in the JSONL file
    for line in f:
        record = json.loads(line)

        # Display the keys of the record to ensure the fields match (optional for debugging)
        print(record.keys())  # You can comment this line after confirming fields are correct

        # Extract relevant fields and prepare them for insertion
        values_to_insert = (
            record.get('mfcc1_sma3Mean', None),
            record.get('mfcc2_sma3Mean', None),
            record.get('mfcc3_sma3Mean', None),
            record.get('mfcc4_sma3Mean', None),
            record.get('F0semitoneFrom27.5Hz_sma3nzMean', None),
            record.get('F1frequency_sma3nzMean', None),
            record.get('turnText', None),
            ', '.join(record['speaker']) if isinstance(record.get('speaker'), list) else record.get('speaker'),
            record.get('startTime', None),
            record.get('endTime', None),
            record.get('duration', None),
            record.get('mp3url', None),
            record.get('turnCount', None),
            record.get('inferredSpeakerRole', None),
            record.get('inferredSpeakerName', None)
        )

        # if isinstance(values_to_insert[7], list):
        #     values_to_insert[7] = ', '.join(values_to_insert[7])

        # Insert the record into the speaker_turns table
        cursor.execute('''
            INSERT INTO speaker_turns (
                mfcc1_sma3Mean, mfcc2_sma3Mean, mfcc3_sma3Mean, mfcc4_sma3Mean, 
                F0semitoneFrom27_5Hz_sma3nzMean, F1frequency_sma3nzMean, turnText, speaker, 
                startTime, endTime, duration, mp3url, turnCount, inferredSpeakerRole, 
                inferredSpeakerName
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', values_to_insert)

    # Commit the changes to the database
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

In [21]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the query to count the number of entries
cursor.execute('SELECT COUNT(*) FROM speaker_turns')
count = cursor.fetchone()[0]

# Print the count
print(f"Number of entries in speaker_turns: {count}")

# Close the cursor and connection
cursor.close()

Number of entries in speaker_turns: 200000


In [ ]:
import sqlite3
from tabulate import tabulate

# Assuming you have an active connection 'conn' to your database
cursor = conn.cursor()

# Execute the SELECT query to fetch all data from the podcast_episodes table
cursor.execute('SELECT * FROM speaker_turns limit 10')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection when done
cursor.close()
conn.close()

In [40]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.db')

# Query the podcast_episodes table
query = "SELECT * FROM speaker_turns"
df = pd.read_sql_query(query, conn)

# Save the dataframe to a CSV file
df.to_csv('../data/speaker_turns.csv', index=False)

# Close the connection
conn.close()

Summary stats

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Query summary statistics for the 'duration' column
try:
    cursor.execute('''
        SELECT 
            MIN(duration) AS min_duration,
            MAX(duration) AS max_duration,
            AVG(duration) AS avg_duration,
            COUNT(duration) AS count_durations
        FROM speaker_turns
    ''')
    
    # Fetch the results
    stats = cursor.fetchone()
    min_duration = stats[0]
    max_duration = stats[1]
    avg_duration = stats[2]
    count_durations = stats[3]

    # Print the summary statistics
    print("Summary Statistics for 'duration':")
    print(f"Minimum Duration: {min_duration}")
    print(f"Maximum Duration: {max_duration}")
    print(f"Average Duration: {avg_duration}")
    print(f"Total Count of Durations: {count_durations}")

    # Additional: Calculate Median (SQLite doesn't have built-in MEDIAN)
    cursor.execute('''
        SELECT duration 
        FROM speaker_turns 
        WHERE duration IS NOT NULL 
        ORDER BY duration
    ''')
    durations = [row[0] for row in cursor.fetchall()]
    n = len(durations)
    
    if n > 0:
        median_duration = (
            durations[n // 2] if n % 2 != 0 else (durations[n // 2 - 1] + durations[n // 2]) / 2
        )
        print(f"Median Duration: {median_duration}")
    else:
        print("Median Duration: No data available")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import pandas as pd

# Read the CSV file
speaker_turns_df = pd.read_csv('../data/speaker_turns.csv')

# Get summary statistics of the duration column
duration_stats = speaker_turns_df['duration'].describe()

# Print the summary statistics
print(duration_stats)

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Query to count occurrences of each inferredSpeakerRole
try:
    cursor.execute('''
        SELECT 
            inferredSpeakerRole, 
            COUNT(*) AS role_count 
        FROM speaker_turns
        WHERE inferredSpeakerRole IS NOT NULL
        GROUP BY inferredSpeakerRole
        ORDER BY role_count DESC
    ''')
    
    # Fetch the results
    role_counts = cursor.fetchall()

    # Print the summary statistics for inferredSpeakerRole
    print("Summary Statistics for 'inferredSpeakerRole':")
    print(f"{'Role':<30} {'Count':<10}")
    print("-" * 40)
    for role, count in role_counts:
        print(f"{role:<30} {count:<10}")
    
    # Additional: Total unique roles
    print("\nTotal Unique Roles:", len(role_counts))

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Query to count occurrences of each inferredSpeakerName
try:
    cursor.execute('''
        SELECT 
            inferredSpeakerName, 
            COUNT(*) AS role_count 
        FROM speaker_turns
        WHERE inferredSpeakerName IS NOT NULL
        GROUP BY inferredSpeakerName
        ORDER BY role_count DESC
    ''')
    
    # Fetch the results
    role_counts = cursor.fetchall()

    # Print the summary statistics for inferredSpeakerName
    print("Summary Statistics for 'inferredSpeakerName':")
    print(f"{'Role':<30} {'Count':<10}")
    print("-" * 40)
    for role, count in role_counts:
        print(f"{role:<30} {count:<10}")
    
    # Additional: Total unique roles
    print("\nTotal Unique Names:", len(role_counts))

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3
import re

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Add helper functions for word and sentence counting
def count_words(text):
    return len(re.findall(r'\b\w+\b', text)) if text else 0

def count_sentences(text):
    return len(re.findall(r'[.!?]', text)) if text else 0

try:
    # Fetch all turnText from the table
    cursor.execute('SELECT turnText FROM speaker_turns WHERE turnText IS NOT NULL')
    turn_texts = cursor.fetchall()

    # Initialize lists to store the counts
    char_counts = []
    word_counts = []
    sentence_counts = []

    # Process each text
    for (text,) in turn_texts:
        char_counts.append(len(text))  # Count characters
        word_counts.append(count_words(text))  # Count words
        sentence_counts.append(count_sentences(text))  # Count sentences

    # Calculate summary statistics for each metric
    def summary_stats(data):
        return {
            "min": min(data),
            "max": max(data),
            "avg": sum(data) / len(data) if data else 0,
            "median": sorted(data)[len(data) // 2] if len(data) % 2 != 0 else (sorted(data)[len(data) // 2 - 1] + sorted(data)[len(data) // 2]) / 2,
            "count": len(data)
        }

    # Compute stats
    char_stats = summary_stats(char_counts)
    word_stats = summary_stats(word_counts)
    sentence_stats = summary_stats(sentence_counts)

    # Print the results
    print("Summary Statistics for 'turnText':\n")

    print("Character Counts:")
    for stat, value in char_stats.items():
        print(f"  {stat.capitalize()}: {value}")

    print("\nWord Counts:")
    for stat, value in word_stats.items():
        print(f"  {stat.capitalize()}: {value}")

    print("\nSentence Counts:")
    for stat, value in sentence_stats.items():
        print(f"  {stat.capitalize()}: {value}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3
import re

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Add helper functions for word and sentence counting
def count_words(text):
    return len(re.findall(r'\b\w+\b', text)) if text else 0

def count_sentences(text):
    return len(re.findall(r'[.!?]', text)) if text else 0

try:
    # Fetch all transcript from the table
    cursor.execute('SELECT transcript FROM podcast_episodes WHERE transcript IS NOT NULL')
    turn_texts = cursor.fetchall()

    # Initialize lists to store the counts
    char_counts = []
    word_counts = []
    sentence_counts = []

    # Process each text
    for (text,) in turn_texts:
        char_counts.append(len(text))  # Count characters
        word_counts.append(count_words(text))  # Count words
        sentence_counts.append(count_sentences(text))  # Count sentences

    # Calculate summary statistics for each metric
    def summary_stats(data):
        return {
            "min": min(data),
            "max": max(data),
            "avg": sum(data) / len(data) if data else 0,
            "median": sorted(data)[len(data) // 2] if len(data) % 2 != 0 else (sorted(data)[len(data) // 2 - 1] + sorted(data)[len(data) // 2]) / 2,
            "count": len(data)
        }

    # Compute stats
    char_stats = summary_stats(char_counts)
    word_stats = summary_stats(word_counts)
    sentence_stats = summary_stats(sentence_counts)

    # Print the results
    print("Summary Statistics for 'transcript':\n")

    print("Character Counts:")
    for stat, value in char_stats.items():
        print(f"  {stat.capitalize()}: {value}")

    print("\nWord Counts:")
    for stat, value in word_stats.items():
        print(f"  {stat.capitalize()}: {value}")

    print("\nSentence Counts:")
    for stat, value in sentence_stats.items():
        print(f"  {stat.capitalize()}: {value}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Execute the query to count rows containing 'George Floyd' in the transcript column
    cursor.execute("""
        SELECT COUNT(*) 
        FROM podcast_episodes 
        WHERE transcript LIKE '%George Floyd%'
    """)
    count = cursor.fetchone()[0]

    # Print the count
    print(f"Number of podcast episodes containing 'George Floyd' in the transcript: {count}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Execute the query to count rows containing 'George Floyd' in the transcript column
    cursor.execute("""
        SELECT COUNT(*) 
        FROM podcast_episodes 
        WHERE transcript LIKE '%black lives matter%'
    """)
    count = cursor.fetchone()[0]

    # Print the count
    print(f"Number of podcast episodes containing 'Black Lives Matter' in the transcript: {count}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Fetch all durationSeconds values
    cursor.execute("SELECT durationSeconds FROM podcast_episodes WHERE durationSeconds IS NOT NULL")
    durations = cursor.fetchall()

    # Convert all values to floats, ignoring any non-numeric ones
    cleaned_durations = []
    for row in durations:
        try:
            cleaned_durations.append(float(row[0]))  # Convert to float
        except ValueError:
            print(f"Skipping invalid value: {row[0]}")  # Handle invalid entries gracefully

    if cleaned_durations:
        # Compute summary statistics
        def summary_stats(data):
            return {
                "min": min(data),
                "max": max(data),
                "avg": sum(data) / len(data) if data else 0,
                "median": sorted(data)[len(data) // 2] if len(data) % 2 != 0 else (sorted(data)[len(data) // 2 - 1] + sorted(data)[len(data) // 2]) / 2,
                "count": len(data)
            }

        stats = summary_stats(cleaned_durations)

        # Print results
        print("Summary Statistics for 'durationSeconds':\n")
        for key, value in stats.items():
            print(f"  {key.capitalize()}: {value}")
    else:
        print("No valid numeric data found in the 'durationSeconds' column.")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Fetch all avgTurnDuration values
    cursor.execute("SELECT avgTurnDuration FROM podcast_episodes WHERE avgTurnDuration IS NOT NULL")
    durations = cursor.fetchall()

    # Convert all values to floats, ignoring any non-numeric ones
    cleaned_durations = []
    for row in durations:
        try:
            cleaned_durations.append(float(row[0]))  # Convert to float
        except ValueError:
            print(f"Skipping invalid value: {row[0]}")  # Handle invalid entries gracefully

    if cleaned_durations:
        # Compute summary statistics
        def summary_stats(data):
            return {
                "min": min(data),
                "max": max(data),
                "avg": sum(data) / len(data) if data else 0,
                "median": sorted(data)[len(data) // 2] if len(data) % 2 != 0 else (sorted(data)[len(data) // 2 - 1] + sorted(data)[len(data) // 2]) / 2,
                "count": len(data)
            }

        stats = summary_stats(cleaned_durations)

        # Print results
        print("Summary Statistics for 'avgTurnDuration':\n")
        for key, value in stats.items():
            print(f"  {key.capitalize()}: {value}")
    else:
        print("No valid numeric data found in the 'avgTurnDuration' column.")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Query to count occurrences of each podTitle
try:
    cursor.execute('''
        SELECT 
            podTitle, 
            COUNT(*) AS pod_count 
        FROM podcast_episodes
        WHERE podTitle IS NOT NULL
        GROUP BY podTitle
        ORDER BY pod_count DESC
    ''')
    
    # Fetch the results
    role_counts = cursor.fetchall()

    # Print the summary statistics for podTitle
    print("Summary Statistics for 'podTitle':")
    print(f"{'Podcast':<30} {'Count':<10}")
    print("-" * 40)
    for role, count in role_counts:
        print(f"{role:<30} {count:<10}")
    
    # Additional: Total unique roles
    print("\nTotal Unique Names:", len(role_counts))

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

Action-infused sentences from transcripts

In [ ]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm

# Load the podcast data
target_df = pd.read_csv("../data/podcast_episodes.csv")
output_file = "../data/podcast_episodes_action_focused.csv"

# Load the action dictionary
action_dic = pd.read_csv("../extract_collective_action/data/collective_action_dic.csv", names=['action'])

def match_word(word, word_list):
    """
    Matches a word with a list of words, supporting wildcards (`*`).
    """
    count = 0
    for w in word_list:
        if word.endswith("*") and w.startswith(word[:-1]):
            count += 1
        elif word == w:
            count += 1
    return count

def get_action_sents(text):
    """
    Extracts the most action-infused sentence(s) from the given text.
    """
    if not isinstance(text, str) or not text.strip():
        return ""
    
    sents = sent_tokenize(text)
    if not sents:
        return ""

    list_sents = []
    for sent in sents:
        action_count = 0
        word_list_sent = word_tokenize(sent)
        for word in action_dic['action']:
            action_count += match_word(word, word_list_sent)
        list_sents.append([sent, action_count / max(len(word_list_sent), 1)])

    # Find the most action-infused sentence
    max_action_sent = max(list_sents, key=lambda x: x[1])
    max_action_sent_idx = list_sents.index(max_action_sent)
    
    # Construct surrounding sentence context
    if len(list_sents) == 1:
        text = max_action_sent[0]
    elif max_action_sent_idx == 0:
        text = " ".join([max_action_sent[0], list_sents[max_action_sent_idx + 1][0]])
    elif max_action_sent_idx == len(list_sents) - 1:
        text = " ".join([list_sents[max_action_sent_idx - 1][0], max_action_sent[0]])
    else:
        text = " ".join([
            list_sents[max_action_sent_idx - 1][0],
            max_action_sent[0],
            list_sents[max_action_sent_idx + 1][0]
        ])

    return text

# # Process transcripts and collect results
# list_texts_tosave = []
# for _, row in tqdm(target_df.iterrows(), total=target_df.shape[0]):
#     text = row['transcript']
#     if not isinstance(text, str) or not text.strip():
#         continue

#     action_count = sum(match_word(word, word_tokenize(text)) for word in action_dic['action'])

#     # Skip transcripts with insufficient action-related words
#     if action_count < 2:
#         continue

#     action_text = get_action_sents(text)
#     list_texts_tosave.append([
#         row['epTitle'], row['podTitle'], row['transcript'], action_text
#     ])

# # Save results to CSV
# df = pd.DataFrame(list_texts_tosave, columns=['epTitle', 'podTitle', 'OriginalTranscript', 'ActionFocusedTranscript'])
# df.to_csv(output_file, index=False)

# print(f"Processed data saved to {output_file}")

# Process transcripts and collect results
list_texts_tosave = []
for _, row in tqdm(target_df.iterrows(), total=target_df.shape[0]):
    text = row['transcript']
    
    # Check for valid text before proceeding
    if not isinstance(text, str) or not text.strip():
        continue

    try:
        # Tokenize the text and count matching action words
        tokenized_text = word_tokenize(text)
        action_count = sum(match_word(word, tokenized_text) for word in action_dic['action'])
    except Exception as e:
        print(f"Error processing text: {e}")
        continue

    # Skip transcripts with insufficient action-related words
    if action_count < 2:
        continue

    # Get the action-focused sentence(s)
    action_text = get_action_sents(text)
    list_texts_tosave.append([
        row['epTitle'], row['podTitle'], row['transcript'], action_text
    ])

# Save results to CSV
df = pd.DataFrame(list_texts_tosave, columns=['epTitle', 'podTitle', 'OriginalTranscript', 'ActionFocusedTranscript'])
df.to_csv(output_file, index=False)

print(f"Processed data saved to {output_file}")

In [65]:
pd.set_option('display.max_colwidth', None)

In [ ]:
podcast_episodes_action_focused = pd.read_csv("../data/podcast_episodes_action_focused.csv")

In [ ]:
# # Process transcripts and collect results
# list_texts_tosave = []
# for _, row in tqdm(target_df.iterrows(), total=target_df.shape[0]):
#     text = row['transcript']
    
#     # Check for valid text before proceeding
#     if not isinstance(text, str) or not text.strip():
#         continue

#     try:
#         # Tokenize the text and count matching action words
#         tokenized_text = word_tokenize(text)
#         action_count = sum(match_word(word, tokenized_text) for word in action_dic['action'])
#     except Exception as e:
#         print(f"Error processing text: {e}")
#         continue

#     # Skip transcripts with insufficient action-related words
#     if action_count < 2:
#         continue

#     # Get the action-focused sentence(s)
#     action_text = get_action_sents(text)
#     list_texts_tosave.append([
#         row['epTitle'], row['podTitle'], row['transcript'], action_text
#     ])

# # Save results to CSV
# df = pd.DataFrame(list_texts_tosave, columns=['epTitle', 'podTitle', 'OriginalTranscript', 'ActionFocusedTranscript'])
# df.to_csv(output_file, index=False)

# print(f"Processed data saved to {output_file}")

In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# Set device to CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_name = "ariannap22/collectiveaction_roberta_simplified_synthetic_weights"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load data
input_file = "../data/podcast_episodes_action_focused.csv"
output_file = "../data/podcast_episodes_action_predictions.csv"
target_df = pd.read_csv(input_file)

# Ensure the ActionFocusedTranscript column is present
if "ActionFocusedTranscript" not in target_df.columns:
    raise ValueError("The column 'ActionFocusedTranscript' was not found in the input file.")

# Prepare texts for prediction
texts = target_df["ActionFocusedTranscript"].dropna().tolist()  # Drop NaNs

# Tokenize and process texts in batches for better memory efficiency
batch_size = 16
predictions = []
probabilities = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i + batch_size]
    inputs = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    # Perform prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)

    # Store predictions and probabilities
    predicted_class_indices = torch.argmax(probs, dim=-1)
    predictions.extend(predicted_class_indices.cpu().tolist())
    probabilities.extend(probs.cpu().tolist())

# Append predictions and probabilities to DataFrame
target_df["PredictedClass"] = pd.Series(predictions, index=target_df.index[:len(predictions)])
target_df["PredictionProbabilities"] = pd.Series(probabilities, index=target_df.index[:len(probabilities)])

# Save the updated DataFrame to CSV
target_df.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")

In [ ]:
podcast_episodes_action_predictions = pd.read_csv("../data/podcast_episodes_action_predictions.csv")

### Topic model

In [ ]:
# import sqlite3
# import pandas as pd

# conn = sqlite3.connect('data.db')

# query = "SELECT epDescription, category1, category2, category3, category4, category5, category6, category7, category8, category9, category10 FROM podcast_episodes WHERE epDescription IS NOT NULL"
# df = pd.read_sql_query(query, conn)
# conn.close()

# print(f"Loaded {len(df)} episode descriptions.")

In [146]:
# import re
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# stop_words = set(stopwords.words('english'))

# def preprocess_text(text):
#     # Remove non-alphanumeric characters and lowercase
#     text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
#     # Tokenize and remove stopwords
#     tokens = word_tokenize(text)
#     return [word for word in tokens if word not in stop_words and len(word) > 2]

# # Apply preprocessing
# df['tokens'] = df['epDescription'].apply(preprocess_text)

In [ ]:
import sqlite3
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Set up stopwords
stop_words = set(stopwords.words('english'))

# Function to preprocess the text
def preprocess_text(text):
    # Remove <p> at the beginning and </p> at the end (if present)
    text = re.sub(r'^<p>', '', text)  # Remove <p> at the start
    text = re.sub(r'</p>$', '', text)  # Remove </p> at the end
    # Remove non-alphanumeric characters and lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    return [word for word in tokens if word not in stop_words and len(word) > 2]

# Connect to the SQLite database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Add 'tokens' column if it does not exist
try:
    cursor.execute("ALTER TABLE podcast_episodes ADD COLUMN tokens TEXT")
except sqlite3.OperationalError:
    # If the column already exists, this will be ignored
    pass

# Fetch the relevant data from the database
query = "SELECT rowid, epDescription FROM podcast_episodes WHERE epDescription IS NOT NULL"
cursor.execute(query)
episodes = cursor.fetchall()

print(f"Loaded {len(episodes)} episode descriptions.")

# Process each episode description and update the table
for row in episodes:
    rowid, ep_description = row
    tokens = preprocess_text(ep_description)
    tokenized_text = ' '.join(tokens)  # Convert list of tokens into a single string
    
    # Update the database with the tokenized text
    update_query = "UPDATE podcast_episodes SET tokens = ? WHERE rowid = ?"
    cursor.execute(update_query, (tokenized_text, rowid))

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f"Tokenized {len(episodes)} episode descriptions.")

In [ ]:
import sqlite3
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Set up stopwords
stop_words = set(stopwords.words('english'))

# Function to preprocess the text
def preprocess_text(text):
    # Remove <p> at the beginning and </p> at the end (if present)
    text = re.sub(r'^<p>', '', text)  # Remove <p> at the start
    text = re.sub(r'</p>$', '', text)  # Remove </p> at the end
    # Remove non-alphanumeric characters and lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Tokenize and remove stopwords
    tokens_transcript = word_tokenize(text)
    return [word for word in tokens_transcript if word not in stop_words and len(word) > 2]

# Connect to the SQLite database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Add 'tokens_transcript' column if it does not exist
try:
    cursor.execute("ALTER TABLE podcast_episodes ADD COLUMN tokens_transcript TEXT")
except sqlite3.OperationalError:
    # If the column already exists, this will be ignored
    pass

# Fetch the relevant data from the database
query = "SELECT rowid, transcript FROM podcast_episodes WHERE transcript IS NOT NULL"
cursor.execute(query)
episodes = cursor.fetchall()

print(f"Loaded {len(episodes)} episode transcripts.")

# Process each episode transcript and update the table
for row in episodes:
    rowid, ep_transcript = row
    tokens_transcript = preprocess_text(ep_transcript)
    tokenized_text = ' '.join(tokens_transcript)  # Convert list of tokens_transcript into a single string
    
    # Update the database with the tokenized text
    update_query = "UPDATE podcast_episodes SET tokens_transcript = ? WHERE rowid = ?"
    cursor.execute(update_query, (tokenized_text, rowid))

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f"Tokenized {len(episodes)} episode transcripts.")

In [3]:
import sqlite3
import numpy as np
from gensim import corpora, models

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Fetch the tokens from the database
cursor.execute("SELECT rowid, tokens FROM podcast_episodes WHERE tokens IS NOT NULL")
rows = cursor.fetchall()

# Prepare the corpus from tokens
texts = []
row_ids = []

for row in rows:
    row_id, token_str = row
    # Convert the stored token strings back to lists (assuming tokens were stored as space-separated text)
    tokens = token_str.split()
    texts.append(tokens)
    row_ids.append(row_id)

# Create dictionary and corpus for LDA
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the LDA model
num_topics = 200
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Get the dominant topic for each episode
dominant_topics = []
for bow in corpus:
    topic_dist = lda_model.get_document_topics(bow)
    if topic_dist:
        dominant_topic = max(topic_dist, key=lambda x: x[1])[0]  # Topic with highest weight
        dominant_topics.append(dominant_topic)
    else:
        dominant_topics.append(None)

# # Ensure the 'topic' column exists in the table
# cursor.execute("PRAGMA table_info(podcast_episodes);")
# columns = [info[1] for info in cursor.fetchall()]
# if 'topic' not in columns:
#     print("Adding 'topic' column to the table.")
#     cursor.execute("ALTER TABLE podcast_episodes ADD COLUMN topic INTEGER")

# Print top words for each topic
for i in range(num_topics):
    words = lda_model.show_topic(i, topn=10)
    top_words = ", ".join([word for word, _ in words])
    print(f"Topic {i}: {top_words}")

# Update the database with the dominant topic
for row_id, topic in zip(row_ids, dominant_topics):
    if topic is not None:
        cursor.execute("UPDATE podcast_episodes SET topic = ? WHERE rowid = ?", (topic, row_id))

# Commit and close
conn.commit()
conn.close()

print(f"Updated {len(dominant_topics)} episodes with their dominant topics.")

Topic 0: see, privacy, informationlearn, upcoming, pleasure, shares, toward, los, artist, salt
Topic 1: enjoy, please, mind, consider, peace, episode, bring, focus, discuss, life
Topic 2: design, anonymous, wide, brain, andy, lots, mediaa, podcast, comfort, website
Topic 3: track, war, none, jordan, campaign, idea, diversity, slide, randy, needed
Topic 4: host, businesses, various, last, episode, publishing, profit, ashley, tactics, strategist
Topic 5: light, exercise, eating, solo, heal, body, performing, disorder, caf, evening
Topic 6: church, pastor, classpspan, sermon, west, matthew, fascinating, chapel, conversation, wersquore
Topic 7: many, tell, taking, shared, challenges, opportunities, times, areas, challenging, people
Topic 8: new, head, york, space, june, student, force, news, students, central
Topic 9: sound, fans, reopening, eternal, israel, exist, court, return, facilities, guided
Topic 10: went, mother, beautiful, california, brown, mom, mothers, daughter, hair, gray
Top

In [26]:
import sqlite3
import numpy as np
from gensim import corpora, models

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Fetch the tokens from the database
cursor.execute("SELECT rowid, tokens_transcript FROM podcast_episodes WHERE tokens_transcript IS NOT NULL")
rows = cursor.fetchall()

# Prepare the corpus from tokens
texts = []
row_ids = []

for row in rows:
    row_id, token_str = row
    tokens = token_str.split()
    texts.append(tokens)
    row_ids.append(row_id)

# Create dictionary and corpus for LDA
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the LDA model
num_topics = 200
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Get the dominant topic for each episode
dominant_topics = []
for bow in corpus:
    topic_dist = lda_model.get_document_topics(bow)
    if topic_dist:
        dominant_topic = max(topic_dist, key=lambda x: x[1])[0]  # Topic with highest weight
        dominant_topics.append(dominant_topic)
    else:
        dominant_topics.append(None)

# Ensure the 'topic' column exists in the table
cursor.execute("PRAGMA table_info(podcast_episodes);")
columns = [info[1] for info in cursor.fetchall()]
if 'topic_transcript' not in columns:
    print("Adding 'topic_transcript' column to the table.")
    cursor.execute("ALTER TABLE podcast_episodes ADD COLUMN topic_transcript INTEGER")

# Print top words for each topic
for i in range(num_topics):
    words = lda_model.show_topic(i, topn=10)
    top_words = ", ".join([word for word, _ in words])
    print(f"Transcript topic {i}: {top_words}")

# Update the database with the dominant topic
for row_id, topic in zip(row_ids, dominant_topics):
    if topic is not None:
        cursor.execute("UPDATE podcast_episodes SET topic_transcript = ? WHERE rowid = ?", (topic, row_id))

# Commit and close
conn.commit()
conn.close()

print(f"Updated {len(dominant_topics)} episodes with their dominant transcript topics.")

Transcript topic 0: know, like, people, youre, one, think, really, thats, dont, right
Transcript topic 1: know, like, yeah, thats, going, think, dont, youre, right, get
Transcript topic 2: timothy, philip, artemis, athens, greek, thessalonians, corinth, lamentations, macedonia, phillip
Transcript topic 3: gulf, naturopathic, like, thats, know, think, dont, yeah, going, youre
Transcript topic 4: new, weve, time, long, think, talking, camera, fancy, service, practicing
Transcript topic 5: queen, athletics, abish, ammon, humphreys, lamoni, wiggins, like, know, dont
Transcript topic 6: like, know, thats, yeah, really, one, right, dont, think, would
Transcript topic 7: macros, perfectionism, selfimposed, henrik, jig, fugue, every, youre, get, know
Transcript topic 8: going, know, like, think, thats, yeah, dont, one, youre, get
Transcript topic 9: like, know, yeah, one, think, dont, thats, really, people, going
Transcript topic 10: people, one, would, god, chapter, king, see, book, israel, m

In [ ]:
# import sqlite3

# # Connect to the database
# db_path = "data.db"
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Update the values in the BLM column where topic is 26 or 51
# update_query = """
# UPDATE podcast_episodes
# SET BLM = 1
# WHERE topic IN (26, 51);
# """

# # Execute the query and commit changes
# cursor.execute(update_query)
# conn.commit()

# # Print the number of rows updated
# print(f"{cursor.rowcount} rows updated.")

# # Close the connection
# conn.close()

In [14]:
import sqlite3
from datetime import datetime

# Connect to SQLite3 database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Query to fetch date columns
query = '''
SELECT rowid, lastUpdate, createdOn, episodeDateLocalized FROM podcast_episodes
'''

rows = cursor.execute(query).fetchall()

for row in rows:
    rowid, lastUpdate, createdOn, episodeDateLocalized = row

    def convert_timestamp(ts):
        if ts is None:
            return None
        try:
            # Handle potential millisecond timestamps by checking value size
            if ts > 1e10:  # Likely milliseconds
                ts = ts / 1000
            return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        except (ValueError, TypeError):
            return None

    # Convert timestamps with error handling
    formatted_lastUpdate = convert_timestamp(lastUpdate)
    formatted_createdOn = convert_timestamp(createdOn)
    formatted_episodeDateLocalized = convert_timestamp(episodeDateLocalized)

    # Update the row with readable dates
    cursor.execute('''
        UPDATE podcast_episodes 
        SET lastUpdate = ?, createdOn = ?, episodeDateLocalized = ? 
        WHERE rowid = ?
    ''', (formatted_lastUpdate, formatted_createdOn, formatted_episodeDateLocalized, rowid))

# Commit and close connection
conn.commit()
conn.close()

In [110]:
import sqlite3
from tabulate import tabulate

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the SELECT query to fetch the first 10 rows from the podcast_episodes table
cursor.execute('SELECT podTitle, epTitle, createdOn, lastUpdate, episodeDateLocalized, epDescription FROM podcast_episodes WHERE description_topic = 148')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection
cursor.close()
conn.close()

+----------------------------------+--------------------------------------------------------+---------------------+---------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|             podTitle             |                        epTitle                         |      createdOn      |     lastUpdate      | episodeDateLocalized |                                                                                                                                                          

In [112]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the query to count the number of rows with description_topic = 148
query = "SELECT COUNT(*) FROM podcast_episodes WHERE description_topic = 122"
cursor.execute(query)

# Fetch the result
count = cursor.fetchone()[0]
print(f"Number of rows with description_topic = 148: {count}")

# Close the cursor and the connection
cursor.close()
conn.close()

Number of rows with description_topic = 148: 282


In [72]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Define the columns to analyze
category_columns = [
    'category1', 'category2', 'category3', 'category4', 'category5',
    'category6', 'category7', 'category8', 'category9', 'category10'
]

# Initialize a dictionary to store the results
category_stats = {}

# Fetch distinct categories and their counts for each column
for column in category_columns:
    query = f'''
    SELECT {column}, COUNT(*) as count
    FROM podcast_episodes
    WHERE {column} IS NOT NULL
    GROUP BY {column}
    ORDER BY count DESC
    '''
    cursor.execute(query)
    results = cursor.fetchall()
    category_stats[column] = results

# Close the cursor and connection
cursor.close()
conn.close()

# Convert the results to a DataFrame
category_stats_df = pd.DataFrame([
    {'Category': category, 'Column': column, 'Count': count}
    for column, values in category_stats.items()
    for category, count in values
])

# Save the DataFrame to a CSV file
category_stats_df.to_csv('category_stats.csv', index=False)

print("Category statistics saved to category_stats.csv")

Category statistics saved to category_stats.csv


In [85]:
import pandas as pd

# Initialize a dictionary to store the count of unique categories for each column
category_summary = {}

# Iterate through category1 to category8 and count unique categories
for category in category_columns[:8]:  # Only consider category1 to category8
    unique_categories = len(category_stats[category])
    category_summary[category] = unique_categories

# Convert the summary dictionary to a DataFrame for better visualization
category_summary_df = pd.DataFrame(list(category_summary.items()), columns=['Category Column', 'Unique Categories Count'])

# Print the summary table
category_summary_df

,Category Column,Unique Categories Count
0,category1,21
1,category2,76
2,category3,75
3,category4,68
4,category5,58
5,category6,42
6,category7,26
7,category8,5


In [79]:
import pandas as pd

# Initialize a dictionary to store the top 10 categories for each category column
top_categories = {}

# Iterate through category1 to category8
for category in category_columns[:8]:
    # Get the top 10 most common categories for the current category column
    top_10 = category_stats[category][:10]
    top_categories[category] = top_10

# Convert the dictionary to a DataFrame for better visualization
top_categories_df = pd.DataFrame([
    {'Category Column': column, 'Category': category, 'Count': count}
    for column, values in top_categories.items()
    for category, count in values
])

# Print the DataFrame
top_categories_df

,Category Column,Category,Count
0,category1,religion,1982
1,category1,business,1153
2,category1,society,1118
3,category1,education,977
4,category1,sports,901
...,...,...,...
70,category8,NULL,9987
71,category8,games,8
72,category8,personal,3
73,category8,relationships,1


In [81]:
import pandas as pd

# Set the display option to show all rows
pd.set_option('display.max_rows', None)

# Display the entire DataFrame
top_categories_df

,Category Column,Category,Count
0,category1,religion,1982
1,category1,business,1153
2,category1,society,1118
3,category1,education,977
4,category1,sports,901
5,category1,health,779
6,category1,news,496
7,category1,comedy,490
8,category1,arts,411
9,category1,music,375


In [11]:
from datetime import datetime

timestamp = 1738207428
readable_date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
print(readable_date)

2025-01-30 03:23:48


In [127]:
# from gensim import corpora, models

# # Create a dictionary and corpus
# dictionary = corpora.Dictionary(df['tokens'])
# corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# # Train the LDA model
# num_topics = 100
# lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# # Get topic distributions for each episode
# episode_topics = [lda_model.get_document_topics(bow) for bow in corpus]

# # Convert to dense representation for visualization
# import numpy as np
# dense_topics = np.zeros((len(episode_topics), num_topics))
# for i, topic_dist in enumerate(episode_topics):
#     for topic_id, topic_value in topic_dist:
#         dense_topics[i, topic_id] = topic_value

Merging the two tables

In [1]:
import sqlite3
from tabulate import tabulate

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the SELECT query to fetch the first 10 rows from the podcast_episodes table
cursor.execute('SELECT * FROM podcast_episodes pe JOIN speaker_turns st ON pe.mp3url = st.mp3url LIMIT 10')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection
cursor.close()
conn.close()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [107]:
import pandas as pd
blm_dic = pd.read_csv('../data/racism_list.csv', header=None)
blm_dic.columns = ['keyword']
blm_dic

,keyword
0,racism
1,racial justice
2,police brutality
3,police violence
4,black people
5,black person
6,black lives matter


In [108]:
import sqlite3
import pandas as pd

keywords = blm_dic['keyword'].tolist()  

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Build the dynamic WHERE clause based on the keywords
    where_clause = " OR ".join([f"transcript LIKE '%{keyword}%'" for keyword in keywords])
    query = f"SELECT COUNT(*) FROM podcast_episodes WHERE {where_clause}"

    # Execute the query
    cursor.execute(query)
    count = cursor.fetchone()[0]

    # Print the count
    print(f"Number of podcast episodes containing keywords from the dictionary: {count}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

Number of podcast episodes containing keywords from the dictionary: 1164


In [115]:
import sqlite3

keywords = blm_dic['keyword'].tolist()  

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Build the dynamic WHERE clause based on the keywords
    where_clause = " OR ".join([f"transcript LIKE '%{keyword}%'" for keyword in keywords])
    query = f"SELECT COUNT(DISTINCT category1) FROM podcast_episodes WHERE {where_clause}"

    # Execute the query
    cursor.execute(query)
    unique_category1_count = cursor.fetchone()[0]

    # Print the count of unique category1 values
    print(f"Number of different category1 values: {unique_category1_count}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

Number of different category1 values: 19


In [117]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

keywords = blm_dic['keyword'].tolist()  

# Build the dynamic WHERE clause based on the keywords
where_clause = " OR ".join([f"transcript LIKE '%{keyword}%'" for keyword in keywords])

# Create the query to fetch categories that meet the where clause
query = f'''
SELECT DISTINCT category1
FROM podcast_episodes
WHERE {where_clause}
'''

# Execute the query
cursor.execute(query)

# Fetch all rows from the result
rows = cursor.fetchall()

# Print the categories
for row in rows:
    print(row)

# Close the cursor and the connection
cursor.close()
conn.close()

('education',)
('sports',)
('religion',)
('society',)
('news',)
('health',)
('leisure',)
('arts',)
('tv',)
('comedy',)
('business',)
('music',)
('games',)
('science',)
('history',)
('government',)
('kids',)
('technology',)
('true crime',)


In [120]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

keywords = blm_dic['keyword'].tolist()

# Build the dynamic WHERE clause using safer parameterized queries
where_clause = " OR ".join(["transcript LIKE ?"] * len(keywords))
keyword_patterns = [f"%{keyword}%" for keyword in keywords]

query = f'''
WITH RankedEpisodes AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY category1 ORDER BY rowid) AS rn
    FROM podcast_episodes
    WHERE {where_clause}
)
SELECT * 
FROM RankedEpisodes
WHERE rn = 1
'''

# Execute the query
cursor.execute(query, keyword_patterns)

# Fetch and print the results
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

for row in rows:
    print(dict(zip(column_names, row)))

# Close the cursor and connection
cursor.close()
conn.close()

{'transcript': " Stand safely or not at all What the fuck is up Again and again we back with another scam safely podcast. It's been a minute. I only deal how many I The last one we deal with in May No, the last one we deal with at the end of April Because Drake had just came up Anyways, what the fuck is up we back we've been gone for a minute Back better than ever I really just think like that remote shit. I don't want to do a lot of a lot of remote podcast I like it better You know face to face So I was giving it just a time to breathe The chance to breathe on that bullshit a Lot of shipping going on since the last lot of bullshit since the last podcast we did So first and foremost I want to say thank you for everybody has been patient everybody that has been supporting since the first podcast Everybody that's been retweeting reposting Tagging us putting it in your Insta story Every whatever I ever I every support as we want to thank you. I want to thank you. I appreciate it Like I sa

In [122]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

keywords = blm_dic['keyword'].tolist()
keyword_patterns = [f"%{keyword}%" for keyword in keywords]

# Query to get one episode per unique category
query = f'''
WITH RankedEpisodes AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY category1 ORDER BY rowid) AS rn
    FROM podcast_episodes
    WHERE {" OR ".join(["transcript LIKE ?"] * len(keywords))}
)
SELECT * 
FROM RankedEpisodes
WHERE rn = 1
'''

# Execute the query and fetch the results
cursor.execute(query, keyword_patterns)
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame and export it to CSV
df = pd.DataFrame(rows, columns=column_names)
csv_filename = 'podcast_episodes_export.csv'
df.to_csv(csv_filename, index=False)

print(f"Results successfully exported to {csv_filename}")

# Close the cursor and connection
cursor.close()
conn.close()

Results successfully exported to podcast_episodes_export.csv


In [2]:
pd.set_option('display.max_colwidth', None)

In [4]:
import pandas as pd

test_racial_justice_episodes = pd.read_csv('../data/test_racial_justice_episodes.csv')

test_racial_justice_episodes.drop(columns=['transcript']).head(30)

,rssUrl,epTitle,epDescription,mp3url,podTitle,podDescription,category1,durationSeconds,numUniqueGuests,neitherPredictedNames,numUniqueNeithers,mainEpSpeakers,numMainSpeakers,hostSpeakerLabels,guestSpeakerLabels
0,https://feeds.megaphone.fm/how-to-commit-journalism,Ep. 14: Covering Black Lives Matter Protests,"Four WRAL reporters who covered protests following the death of George Floyd talk about their experience reporting on the peaceful marches, vocal crowds and violent riots, along with the police responses that followed. They discuss what teargas feels like when it hits your eyes, memorable interviews with protesters, and what it's like to cover the destruction of a city you love. The discussion features reporters Aaron Thomas, Amanda Lamb, Bryan Mims and Joe Fisher, along with host Ashley Talley.For WRAL's video coverage of the protests, go to https://www.wral.com/protest/17382846/",https://traffic.megaphone.fm/CAPITOL1150524463.mp3,How To Commit Journalism,"How to Commit Journalism is a podcast that pulls back the curtain on what it takes to report the news. It’s the back story behind the big story, and a look at all that goes into reporting at WRAL News. The news you see on TV and read online is the end product of hours of work by many people: research, interviews, calls, meetings, following stories day in and day out.Each week, WRAL News Executive Producer Ashley Talley sits down with journalists to dig into the intricacies of covering one of the big stories of the week. We’ll talk about the process, the questions and the often difficult decisions that go into what we cover and how we cover it. Occasionally, we’ll also bring you conversations with WRAL journalists about their journey in this profession and the memorable stories they’ve covered.How to Commit Journalism is a production of WRAL News and part of the Capitol Broadcasting podcast network.",NaN,2426.0,4.0,"['Rick Gull', 'George Floyd']",2.0,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE
1,https://anchor.fm/s/127cd708/podcast/rss,Blackout Tuesday,"It’s been a minute since Scam Safely (@scamsafely, @scvmsafely) dropped a new episode, so we decided to provide a fresh perspective on current events . On this episode DaSai & Kris (@_k2times, @gramsandscvms) give you the latest on what’s happening locally and nationally: from the most recent police brutality and racism events to Detroit Villians getting exposed throughout these trying times. The video about Jewell Jones we bring up can be found on @pardonmewithp Instagram page in his Instagram Tv section. Support and shop business. RIP GEORGE FLOYD! www.scamsafely.com--- Send in a voice message: https://podcasters.spotify.com/pod/show/scvmsafely/messageSupport this podcast: <a href='https://podcasters.spotify.com/pod/show/scvmsafely/support' rel='payment'>https://podcasters.spotify.com/pod/show/scvmsafely/support</a>",https://anchor.fm/s/127cd708/podcast/play/14692250/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fstaging%2F2020-06-03%2F7df5746d9c2be320ec38d982dbb8955a.m4a,$CVM Safely,"Whether you paint, draw, design, etc., this is for us! A podcast for creatives & entrepreneurs. Listen to DaSai talk about a array topics with occasional guest bi weekly. You might learn a lot & you might laugh a lot. Support this podcast: <a href=""https://podcasters.spotify.com/pod/show/scvmsafely/support"" rel=""payment"">https://podcasters.spotify.com/pod/show/scvmsafely/support</a>",arts,5301.0,0.0,NO_NEITHER_IDENTIFIED,0.0,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE
2,https://feeds.soundcloud.com/users/soundcloud:users:583416702/sounds.rss,S2EP68- The 2020 Racial War,The racial war in 2020 has been the topic of conversation on various platforms. Join us this week as we share how our thoughts and some tips on how to combat this war alongside the roles we may choose to take. Don't forget to follow us on Instagram at Advice Fr

In [130]:
# Keep only the specified columns in the test_racial_justice_episodes DataFrame
columns_to_keep = [
    'transcript', 'rssUrl', 'epTitle', 'epDescription', 'mp3url',
    'podTitle', 'podDescription', 'category1', 'durationSeconds', 
    'numUniqueGuests', 'neitherPredictedNames', 'numUniqueNeithers',
    'mainEpSpeakers', 'numMainSpeakers', 'hostSpeakerLabels', 'guestSpeakerLabels'
]

test_racial_justice_episodes = test_racial_justice_episodes[columns_to_keep]
test_racial_justice_episodes.head()

,transcript,rssUrl,epTitle,epDescription,mp3url,podTitle,podDescription,category1,durationSeconds,numUniqueGuests,neitherPredictedNames,numUniqueNeithers,mainEpSpeakers,numMainSpeakers,hostSpeakerLabels,guestSpeakerLabels
0,Stand safely or not at all What the fuck is u...,https://anchor.fm/s/127cd708/podcast/rss,Blackout Tuesday,It’s been a minute since Scam Safely (@scamsaf...,https://anchor.fm/s/127cd708/podcast/play/1469...,$CVM Safely,"Whether you paint, draw, design, etc., this is...",arts,5301.0,0.0,NO_NEITHER_IDENTIFIED,0.0,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE
1,"Hey, you click play and I want to welcome you...",https://feeds.soundcloud.com/users/soundcloud:...,S2EP68- The 2020 Racial War,The racial war in 2020 has been the topic of c...,https://feeds.soundcloud.com/stream/840849208-...,Advice From Hanna Podcast,The Advice From Hanna Podcast is a season life...,business,985.0,0.0,NO_NEITHER_IDENTIFIED,0.0,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE
2,(upbeat music) - Yeah. - There'll be no props...,https://anchor.fm/s/1a156ac/podcast/rss,A Fine Print Review of 'The Last Dance' Ep 9.-...,<p>Anthony Maxwell is back to discuss Episodes...,https://anchor.fm/s/1a156ac/podcast/play/14153...,"Read The Fine Print, The Show with DeTravius B...","Join me, DeTravius Bethea, Esq., your host, on...",comedy,4104.0,0.0,"[""Marlon Wayne's"", ""Daniel Wayne's"", 'Kenny La...",5.0,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE
3,"[Music] Welcome to ""Brazing Education"" with E...",https://brazeneducation.podbean.com/feed.xml,BE: Episode 38 - Why Black Educators Feel Emot...,<p>Listen as Educator Barnes explains how rece...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,https://mcdn.podbean.com/mf/web/as0pgw/Why_Bla...,Brazen Education,2023-09-21 09:41:02,Educator Barnes,Educator Barnes,how to,NaN,NO_GUEST_PREDICTED,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE,SPEAKER_DATA_UNAVAILABLE


In [131]:
test_racial_justice_episodes.to_csv('test_racial_justice_episodes.csv', index=False)

In [88]:
import sqlite3
import pandas as pd

keywords = blm_dic['keyword'].tolist()  

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Build the dynamic WHERE clause based on the keywords
    where_clause = " OR ".join([f"transcript LIKE '%{keyword}%'" for keyword in keywords])
    query = f"SELECT SUM(sentences) FROM podcast_episodes WHERE {where_clause}"

    # Execute the query
    cursor.execute(query)
    total_sentences = cursor.fetchone()[0]

    # Print the total sum of sentences
    print(f"Total sum of sentences in podcast episodes containing keywords: {total_sentences}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

Total sum of sentences in podcast episodes containing keywords: 674605


In [89]:
import sqlite3

keywords = blm_dic['keyword'].tolist()  

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

try:
    # Build the dynamic WHERE clause based on the keywords
    where_clause = " OR ".join([f"transcript LIKE '%{keyword}%'" for keyword in keywords])

    # Create the query to join the tables and count rows where turnText is not None
    query = f'''
    SELECT COUNT(*)
    FROM podcast_episodes pe
    LEFT JOIN speaker_turns st ON pe.mp3url = st.mp3url
    WHERE ({where_clause}) AND st.turnText IS NOT NULL
    '''

    # Execute the query
    cursor.execute(query)
    count_with_turnText = cursor.fetchone()[0]

    # Print the count of rows with turnText not None
    print(f"Number of podcast episodes with keywords and non-null turnText: {count_with_turnText}")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

Number of podcast episodes with keywords and non-null turnText: 34591


Checking that topic modeling worked

In [90]:
import sqlite3
from tabulate import tabulate

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the SELECT query to fetch the first 10 rows from the podcast_episodes table
cursor.execute('SELECT transcript_1000_tokens, dominant_topic_number_1000, topic_distribution_1000 FROM podcast_episodes LIMIT 5')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection
cursor.close()
conn.close()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [93]:
# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the query to count the number of rows with dominant_topic_number = ?
query = "SELECT COUNT(*) FROM podcast_episodes WHERE dominant_topic_number_1000 = 97 OR dominant_topic_number_1000 = 193"
cursor.execute(query)

# Fetch the result
count = cursor.fetchone()[0]
print(f"Number of rows with racial justice-related dominant_topic_number_1000: {count}")

# Close the cursor and the connection
cursor.close()
conn.close()

Number of rows with racial justice-related dominant_topic_number_1000: 13


In [94]:
# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the query to count the number of rows with dominant_topic_number = ?
query = "SELECT COUNT(*) FROM podcast_episodes WHERE dominant_topic_number = 38"
cursor.execute(query)

# Fetch the result
count = cursor.fetchone()[0]
print(f"Number of rows with racial justice-related dominant_topic_number: {count}")

# Close the cursor and the connection
cursor.close()
conn.close()

Number of rows with racial justice-related dominant_topic_number: 81


In [99]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the SELECT query to fetch the first 10 rows from the podcast_episodes table
cursor.execute('SELECT epDescription FROM podcast_episodes WHERE dominant_topic_number = 38 AND (dominant_topic_number_1000 = 97 OR dominant_topic_number_1000 = 193)')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection
cursor.close()
conn.close()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Important query

In [70]:
import sqlite3
from tabulate import tabulate

# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

keywords = blm_dic['keyword'].tolist() 

# Build the WHERE clause dynamically based on the keywords
where_clause = " OR ".join([f"transcript LIKE '%{keyword}%'" for keyword in keywords])

query = f'''
SELECT st.turnText, st.inferredSpeakerRole
FROM podcast_episodes pe
INNER JOIN speaker_turns st ON pe.mp3url = st.mp3url
WHERE st.turnText IS NOT NULL AND UPPER(TRIM(st.inferredSpeakerRole)) != UPPER('NO_INFERRED_ROLE') AND ({where_clause})
LIMIT 10
'''
# AND UPPER(TRIM(st.inferredSpeakerRole)) != UPPER('NO_INFERRED_ROLE')


cursor.execute(query)

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the table
column_names = [description[0] for description in cursor.description]

# Print the table using tabulate
print(tabulate(rows, headers=column_names, tablefmt="pretty"))

# Close the cursor and the connection
cursor.close()
conn.close()

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+
|                                                                                                                                                                                               turnText                                                                                                                                                                                                | inferredSpeakerRole |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

Podcast index database (doesn't work)

In [27]:
# import sqlite3
# import pandas as pd

# # Path to the database
# db_path = "podcastindex_feeds.db"  # Ensure this is correct

# # Connect to the SQLite database
# conn = sqlite3.connect(db_path)

# # Query the podcasts table and load it into a DataFrame
# query = "SELECT * FROM podcasts LIMIT 5;"  # Limiting to 5 rows for brevity
# df = pd.read_sql_query(query, conn)

# # Close the connection
# conn.close()

# # Display the DataFrame
# df.head()

In [22]:
import sqlite3

db_path = "podcastindex_feeds.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

conn.close()

Tables in the database: []
